In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import xgboost as xgb

###### Data Preprocessing

In [2]:
dataset = pd.read_csv('./combined_dataset/combined_datasets.csv', error_bad_lines=False)
dataset = dataset[dataset['zipcode'] > 0]
dataset = dataset[dataset['avg'] > 0]
dataset['zipcode'] = dataset['zipcode'].astype(int).astype(str)
dataset.head()

b'Skipping line 18654: expected 8 fields, saw 15\n'


,zipcode,subsystem,lat,long,sensor,parameter,datetime,avg
0,60601,chemsense,41.884607,-87.624577,no2,concentration,5/10/18 19:00,0.006243
1,60612,chemsense,41.871480,-87.676440,at0,temperature,5/1/18 12:00,27.369161
2,60610,chemsense,41.903632,-87.648093,so2,concentration,5/5/18 3:00,0.197254
3,60606,lightsense,41.883260,-87.633870,tmp421,temperature,5/4/18 2:00,39.129429
6,60639,chemsense,41.923996,-87.761072,at3,temperature,5/9/18 14:00,27.192733


In [3]:
dataset['datetime'] = pd.to_datetime(dataset['datetime'])
dataset= dataset[(dataset['datetime'] >= '2018-05-01') & (dataset['datetime'] <= '2018-06-30')]
dataset.head()

,zipcode,subsystem,lat,long,sensor,parameter,datetime,avg
0,60601,chemsense,41.884607,-87.624577,no2,concentration,2018-05-10 19:00:00,0.006243
1,60612,chemsense,41.871480,-87.676440,at0,temperature,2018-05-01 12:00:00,27.369161
2,60610,chemsense,41.903632,-87.648093,so2,concentration,2018-05-05 03:00:00,0.197254
3,60606,lightsense,41.883260,-87.633870,tmp421,temperature,2018-05-04 02:00:00,39.129429
6,60639,chemsense,41.923996,-87.761072,at3,temperature,2018-05-09 14:00:00,27.192733


In [8]:
# encode categorical features
dataset_enc = pd.get_dummies(dataset, drop_first = True)

,lat,long,datetime,avg,zipcode_60601,zipcode_60602,zipcode_60603,zipcode_60604,zipcode_60605,zipcode_60606,...,parameter_pm1_cf1,parameter_pm25_atm,parameter_pm25_cf1,parameter_pm2_5,parameter_point_3um_particle,parameter_point_5um_particle,parameter_pressure,parameter_sample_flow_rate,parameter_sampling_period,parameter_temperature
0,41.884607,-87.624577,2018-05-10 19:00:00,0.006243,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,41.871480,-87.676440,2018-05-01 12:00:00,27.369161,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,41.903632,-87.648093,2018-05-05 03:00:00,0.197254,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41.883260,-87.633870,2018-05-04 02:00:00,39.129429,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
6,41.923996,-87.761072,2018-05-09 14:00:00,27.192733,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,41.903632,-87.648093,2018-05-09 17:00:00,36.878819,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,41.884490,-87.636955,2018-05-09 01:00:00,1492.921429,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
10,41.751142,-87.712990,2018-05-08 18:00:00,3.100476,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
12,41.943263,-87.688069,2018-05-07 09:00:00,63.459286,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,41.916586,-87.666406,2018-05-03 12:00:00,685.082701,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# create training and test sets
